In [1]:
# LLM
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# Vector DB
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_community.document_loaders import JSONLoader

# Output
import os
import pprint as pp
from jinja2 import Template

import sys

sys.path.append(os.path.abspath('..'))
from src.model import Model
from src.contract_parser import ContractParser

root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# root = os.path.abspath(os.path.join(os.getcwd()))

import dotenv
dotenv.load_dotenv(override=True)

root

'c:\\Users\\balia\\OneDrive\\Escritorio\\Facultad\\2Q-2024\\propertyGPT'

In [2]:
import getpass
import os

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")

In [3]:
import json

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["tests"] = json.dumps(record.get("tests"))
    metadata["function"] = record.get("function")
    return metadata

# Reference: https://python.langchain.com/docs/integrations/document_loaders/json
loader = JSONLoader(
    file_path=f"{root}/data/dataset.json",
    jq_schema=".data[]",
    content_key="description",
    metadata_func=metadata_func,
)

docs = loader.load()
len(docs)

17

In [4]:
llm = ChatAnthropic(
    model="claude-3-haiku-20240307",
    # model="claude-3-5-sonnet-20240620",
    temperature=0.8,
)
# llm = ChatOllama(
#     model="qwen2.5-coder:latest",
#     temperature=0.8,
# )

In [13]:
model = Model(docs, test_model=llm, compilation_model=llm, use_rag=False)
# model = Model(docs)

In [14]:
# model.generate_test_function(f"{root}/contracts/ERC20.sol", "ERC20", recompile_tries=0)
model.generate_test_functions(f"{root}/foundry", "Vault", recompile_tries=3, k=1, subtests=1)

**************************************************
Generating Test Function Code
**************************************************
**************************************************
Parsing public or external functions
**************************************************
**************************************************
Generating test functions WITHOUT rag
**************************************************
**************************************************
Function Code with no rag generated
function testContribute() public {
    // Set up test environment
    Vault vault = new Vault();
    address user1 = address(0x1);
    address user2 = address(0x2);
    vm.deal(user1, 1 ether);
    vm.deal(user2, 1 ether);

    // Test contributing less than 0.001 ether
    vm.startPrank(user1);
    vault.contribute{value: 0.0005 ether}();
    assertEq(vault.getContribution(), 0.0005 ether);
    assertEq(vault.owner(), user1);
    vm.stopPrank();

    // Test contributing more than current owner's

In [18]:
model.compiler_errors

Counter({'2314': 1})

In [16]:
model.all_tests

["function testContribute() public {\n    // Set up test environment\n    Vault vault = new Vault();\n    address user1 = address(0x1);\n    address user2 = address(0x2);\n    vm.deal(user1, 1 ether);\n    vm.deal(user2, 1 ether);\n\n    // Test contributing less than 0.001 ether\n    vm.startPrank(user1);\n    vault.contribute{value: 0.0005 ether}();\n    assertEq(vault.getContribution(), 0.0005 ether);\n    assertEq(vault.owner(), user1);\n    vm.stopPrank();\n\n    // Test contributing more than current owner's contribution\n    vm.startPrank(user2);\n    vault.contribute{value: 0.0006 ether}();\n    assertEq(vault.getContribution(), 0.0006 ether);\n    assertEq(vault.owner(), user2);\n    vm.stopPrank();\n}",
 'function testGetContribution() public {\n    Vault vault = new Vault();\n    vault.contribute{value: 0.0005 ether}();\n    assertEq(vault.getContribution(), 0.0005 ether);\n}',
 'function testWithdraw() public {\n    // Arrange\n    Vault vault = new Vault();\n    uint256 in

In [17]:
model.generate_output("Vault", f"{root}/foundry/test")

**************************************************
Generating output
**************************************************
**************************************************
Contract generated in: c:\Users\balia\OneDrive\Escritorio\Facultad\2Q-2024\propertyGPT/foundry/test/Vault.t.sol
**************************************************


In [ ]:
# 1. Usuario pasa path a proyecto Foundry y path a contrato a testear

# 2. Se parsean las funciones del contrato y se testean todas las funciones publicas y externas, de a una.
# NOTA: ignoramos funciones heredadas de otros contratos, por simplicidad. Es una limitacion de la herramienta por ahora.

# 3. Se genera una descripcion de la funcion a testear (usando LLM basico, eg. Llama 3.2) y se hace un similarity search con eso en la vector db con algun K (eg. 3).
# NOTA: se puede poner un threshold de similarity, para no devolver funciones de referencia que sean muy distintas e irrelevantes.

# 4. Por cada funcion de referencia encontrada, se genera un test case con el contrato y la funcion a testear, y se refinan hasta que sean compilables (compilandolos adentro del proyecto Foundry provisto por el usuario).
# Si no llegan a ser compilables, se descartan.

# 5. Se appendean todos los tests a una lista, y se hace lo mismo para todas las funciones del contrato a testear.

# 6. Por ultimo, se genera el template final incluyendo todos los tests y se devuelve al usuario.